In [1]:
import tensorflow
from pysparkling import H2OContext

In [2]:
## Demo-Specific configuration
NODES=3 # Number of partitions to split data into (~simulate a number of nodes)
DATASET_DIR="/users/arno/h2o-3/bigdata/laptop/mnist"

In [3]:
## Initialize TensorFlow session and test it
def map_fun(i):
  import tensorflow as tf
  with tf.Graph().as_default() as g:
    hello = tf.constant('Sparkling, TensorFlow!', name="hello_constant")
    with tf.Session() as sess:
      return sess.run(hello)
sc.parallelize(range(NODES), NODES).map(map_fun).collect()

['Sparkling, TensorFlow!', 'Sparkling, TensorFlow!', 'Sparkling, TensorFlow!']

In [4]:
## Read MNIST data into H2O
import h2o
h2o.__version__
hc = H2OContext(sc).start()
train_frame = h2o.import_file("{}/{}".format(DATASET_DIR, "train.csv.gz"))
test_frame = h2o.import_file("{}/{}".format(DATASET_DIR, "test.csv.gz"))

H2O cluster uptime:,3 seconds 898 milliseconds
H2O cluster version:,3.8.1.3
H2O cluster name:,sparkling-water-arno_-1262711341
H2O cluster total nodes:,1
H2O cluster total free memory:,950.97 MB
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster healthy:,True
H2O Connection ip:,192.168.11.2
H2O Connection port:,54321
H2O Connection proxy:,None



Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%


In [5]:
## Turn H2O DataFrame into a Spark DataFrame
train_df = hc.as_spark_frame(train_frame).repartition(NODES)
test_df = hc.as_spark_frame(test_frame).repartition(NODES)
#train_df.printSchema()

In [10]:
# Configure a TensorFlow Deep Learning model

# - it loads local training data into numpy array (from Spark -> Python)
# - train TF Deep Learning model with 1 hidden layer (50 neurons)
# - output accuracy on training data
def create_nn(data_train, data_test, iterations, batch_size):
    import tensorflow as tf
    # Symbolic computation
    x = tf.placeholder(tf.float32, [None, 784])
    W1 = tf.Variable(tf.random_normal([784, 50],stddev=0.1))
    W2 = tf.Variable(tf.random_normal([50, 10],stddev=0.1))
    b1 = tf.Variable(tf.random_normal([50],stddev=0.1))
    b2 = tf.Variable(tf.random_normal([10],stddev=0.1))
    hidden = tf.nn.relu(tf.matmul(x, W1) + b1)
    y = tf.nn.softmax(tf.matmul(hidden, W2) + b2)
    y_ = tf.placeholder(tf.float32, [None, 10])
    cross_entropy = -tf.reduce_sum(y_*tf.log(y))                    
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    
    # Initialize TF
    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    print("Training TensorFlow Deep Learning model")
    for i in range(iterations):
      #print("TensorFlow iter: ", i, " session: ", sess)
      batch_xs, batch_ys = data_train.next_batch(batch_size)
      sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        
    model = [(sess.run(W1),sess.run(W2),sess.run(b1),sess.run(b2))]

    # Model evaluation
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    batch_xs, batch_ys = data_test.next_batch(1000)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Training Accuracy:", sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys}))
    #print(sess.run(tf.argmax(y,1), feed_dict={x: batch_xs, y_: batch_ys}))
    
    sess.close()
    return iter(model)
    
    # Export the model
    #from tensorflow_serving.session_bundle import exporter
    #export_path = "/tmp/xxx/"
    #saver = tf.train.Saver(sharded=True)
    #model_exporter = exporter.Exporter(saver)
    #signature = exporter.classification_signature(input_tensor=x, scores_tensor=y)
    #model_exporter.init(sess.graph.as_graph_def(), default_graph_signature=signature)
    #model_exporter.export(export_path, tf.constant(FLAGS.export_version), sess)
    
## Internal Helpers

# Sampling with replacement to provide a batch size
# Load everything into numpy datastructure
import numpy as np

def expand1hot(response, levels):
    nrows = response.shape[0]
    result = np.zeros((nrows, levels), dtype=np.float32)
    result[np.arange(nrows), response.astype(np.int8)] = 1.0
    return result

class RowData:
    def __init__(self, it):
        self._part_array = np.array([ [a for a in x] for x in it], dtype=np.float32)
        # Definition of input features
        self._x = range(784)
        # Index of response
        self._y = 784

    def next_batch(self, n):
        # Sample from local data without replacement
        dim = self._part_array.shape[0] # number of rows
        sample = np.random.choice(dim, n, replace=False)
        data = self._part_array[sample, :]
        # Data comming from H2O, pixel values are 0..255
        # FIXME: this should be done via RDD or H2O API directly !
        train = data[:, self._x]/255 
        response = expand1hot(data[:, self._y], 10)
        return (train, response)



In [14]:
## Run TensorFlow on each node

# Number of iterations
ITERATIONS = 100
# Batch size
BATCH_SIZE = 100
# Use Mnist dataset provided by TensorFlow - for debugging only
USE_TF_MNIST=False

def train_nn(iterations, batch_size, use_tf_mnist=False):
    def perPartition(it):
        if not use_tf_mnist:
            train_data = RowData(it)
            test_data = train_data
        else:
            from tensorflow.examples.tutorials.mnist import input_data
            mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
            train_data = mnist.train
            test_data = mnist.train
            
        return create_nn(train_data, test_data, iterations, batch_size)
        
    return perPartition

modelsPerNode = train_df.mapPartitions(train_nn(ITERATIONS, BATCH_SIZE, USE_TF_MNIST)).collect()

In [15]:
# Average the weights and biases across all node-local models
W1 = modelsPerNode[0][0]
W2 = modelsPerNode[0][1]
b1 = modelsPerNode[0][2]
b2 = modelsPerNode[0][3]

AVERAGE = True

if (AVERAGE):
  for i in range(1,NODES):
    W1 = W1 + modelsPerNode[i][0]
    W2 = W2 + modelsPerNode[i][1]
    b1 = b1 + modelsPerNode[i][2]
    b2 = b2 + modelsPerNode[i][3]

W1 = W1/NODES
W2 = W2/NODES
b1 = b1/NODES
b2 = b2/NODES

#print(W1)
#print(W2)
print(b1)
print(b2)

[-0.02555854 -0.04464746 -0.03791011  0.08951918 -0.08080242  0.04897232
  0.15089417 -0.15740228 -0.09470204 -0.11056785 -0.05662239  0.12452243
  0.15506518  0.07133955 -0.08953484  0.03140675 -0.11859402 -0.2315333
  0.02277429 -0.07462133 -0.06297824 -0.04473608  0.01680743 -0.27055848
  0.11561105  0.0200029   0.00805387 -0.08048744 -0.0166826   0.04719077
 -0.12510569 -0.03544705 -0.03799803  0.01852199 -0.03490278 -0.06806166
  0.12192162  0.04496474  0.03271996  0.05014573 -0.01179659 -0.02423133
 -0.0871039   0.02045563  0.03281573 -0.03816571 -0.11163068 -0.13451032
  0.05698619 -0.03011321]
[-0.31255579 -0.29942903 -0.14410944  0.0192977  -0.00965579  0.30137542
 -0.32928514 -0.31712729  0.61885303  0.35101628]


In [16]:
#Initialize an H2O Model with those weights/biases
#from pysparkling import *
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

## Create an H2O Deep Learning model from the TensorFlow model
dlmodel = H2ODeepLearningEstimator(
    hidden=[50],             ## same Network layout as TF - one hidden layer
    epochs=0,                ## no training done in H2O - just copy over the model from TF
    ignore_const_cols=False  ## keep all input features (unless we also drop const cols in TF)
    
    ### Initialize the H2O model with the TensorFlow model state
    ### Requires H2O 3.8.2.1 or later
    #,initial_weights=[h2o.H2OFrame(W1.tolist()),h2o.H2OFrame(W2.tolist())]
    #,initial_biases=[h2o.H2OFrame(b1.tolist()),h2o.H2OFrame(b2.tolist())]
)
train_frame[784] = train_frame[784].asfactor()
dlmodel.train(x=list(range(784)),y=784,training_frame=train_frame)


deeplearning Model Build Progress: [                                                  ] 00%


In [17]:
## Check the model performance - Will only be good if we actually copied the TF model into the H2O model above
dlmodel.model_performance(train_frame)


ModelMetricsMultinomial: deeplearning
** Reported on test data. **

MSE: 0.831983301496
R^2: 0.900334359944
LogLoss: 7.95881104659

Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
382.0,13.0,75.0,66.0,717.0,1736.0,140.0,2793.0,1.0,0.0,0.9355057,"5,541 / 5,923"
424.0,1.0,290.0,192.0,524.0,951.0,76.0,4227.0,56.0,1.0,0.9998517,"6,741 / 6,742"
529.0,34.0,172.0,289.0,889.0,912.0,163.0,2918.0,50.0,2.0,0.9711313,"5,786 / 5,958"
296.0,25.0,149.0,294.0,755.0,1111.0,107.0,3344.0,43.0,7.0,0.9520470,"5,837 / 6,131"
1339.0,157.0,29.0,286.0,720.0,816.0,278.0,2208.0,7.0,2.0,0.8767545,"5,122 / 5,842"
596.0,83.0,64.0,76.0,1038.0,1457.0,152.0,1946.0,4.0,5.0,0.7312304,"3,964 / 5,421"
81.0,75.0,7.0,72.0,409.0,1257.0,1154.0,2855.0,8.0,0.0,0.8050017,"4,764 / 5,918"
1075.0,117.0,205.0,27.0,471.0,231.0,13.0,4074.0,51.0,1.0,0.3497207,"2,191 / 6,265"
203.0,5.0,96.0,74.0,1117.0,2981.0,55.0,1307.0,13.0,0.0,0.9977782,"5,838 / 5,851"
367.0,115.0,104.0,93.0,1059.0,890.0,30.0,3263.0,28.0,0.0,1.0,"5,949 / 5,949"



Top-10 Hit Ratios: 


k,hit_ratio
1,0.1377833
2,0.2441167
3,0.3449666
4,0.4413833
5,0.5369333
6,0.62485
7,0.7107667
8,0.8008
9,0.90955
10,1.0


In [18]:
## Now we have a POJO for the TensorFlow model!
dlmodel.download_pojo()

Filepath: /DeepLearning_model_python_1460344722771_1.java
/*
  Licensed under the Apache License, Version 2.0
    http://www.apache.org/licenses/LICENSE-2.0.html

  AUTOGENERATED BY H2O at 2016-04-10T20:22:06.242-07:00
  3.8.1.3
  
  Standalone prediction code with sample test data for DeepLearningModel named DeepLearning_model_python_1460344722771_1

  How to download, compile and execute:
      mkdir tmpdir
      cd tmpdir
      curl http://192.168.11.2:54321/3/h2o-genmodel.jar > h2o-genmodel.jar
      curl http://192.168.11.2:54321/3/Models.java/DeepLearning_model_python_1460344722771_1 > DeepLearning_model_python_1460344722771_1.java
      javac -cp h2o-genmodel.jar -J-Xmx2g -J-XX:MaxPermSize=128m DeepLearning_model_python_1460344722771_1.java

     (Note:  Try java argument -XX:+PrintCompilation to show runtime JIT compiler behavior.)
*/
import java.util.Map;
import hex.genmodel.GenModel;
import hex.genmodel.annotations.ModelPojo;

@ModelPojo(name="DeepLearning_model_python_146034